# **Set Env**

In [1]:
#安裝跑 PySpark 的環境
!apt-get -y install openjdk-8-jre-headless
!pip install -q pyspark

#install Apache Spark 3.1.2 with Hadoop 2.7 from here.
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#we just need to unzip that folder.
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
# findspark library will locate Spark on the system and import it as a regular library.

!pip install -q findspark
#set the environment path. This will enable us to run Pyspark in the Colab environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


#We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

import findspark
findspark.init()
#know the location where Spark is installed, use findspark.find()
findspark.find()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 2s (13.8 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.

'/content/spark-3.1.2-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# yarn mode
#spark = SparkSession\
#        .builder\
#        .master("yarn")\
#        .config('spark.executor.instances','99')\
#        .config('spark.executor.memory','4G')\
#        .appName("trainRF")\
#        .getOrCreate()

In [3]:
# Check spark app name
spark.sparkContext.appName

'Colab'

In [4]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [5]:
!pip install databricks
!pip install koalas

     |████████████████████████████████| 390 kB 5.3 MB/s 


In [6]:
# need export ARROW_PRE_0_15_IPC_FORMAT=1 in .bashrc
import databricks.koalas as ks

In [7]:
ks.set_option("compute.default_index_type", "distributed")

In [8]:
spark

In [9]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()
# 使用Word Count範例程式，驗證PySpark環境是否可以正常運作
rdd = sc.parallelize(["Hello Spark"])
counts = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
print(counts)

[('Hello', 1), ('Spark', 1)]


In [10]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:46:13Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [11]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-11-01 09:41:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  37.0MB/s    in 0.4s    

2021-11-01 09:41:23 (37.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


# **Start**

In [28]:
import pandas as pd
from pandas import read_csv
import numpy
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorAssembler

In [13]:
# 3.2才可用
#import pyspark.pandas as pd
#from pyspark.pandas import read_csv
# 沒用到pyspark
import sklearn 
from xgboost import XGBClassifier #scikit learn API https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# from sklearn.feature_selection import SequentialFeatureSelector 會報錯 不支援sklearn 0.24
from sklearn.model_selection import train_test_split

In [14]:
df = spark.read.csv('/content/drive/MyDrive/project/data/train90.csv', header=True, inferSchema=True)
df.createOrReplaceTempView("dfTable")  # 為了可執行sql command
df.show(5)

+---+-------------------+----------------------------------+------------------------------+-------------------+------------------+-------------------------+-------------------+------------------+-------------------------+-----------------------+------------------------+-----------------------+-----------------------+------------------------+-----------+-------------------+---------------------+------------+-----------------------+----------------------+------------------------+-------------------------+----------------------+------------------------+----------------------+------------------------+--------------------+-------------------+----------------------+---------------------+-------------------+---------------------+-------------------+---------------------+----------------+----------------+---------------------+------------------------+---------------------------------+------------------+-------------------------+-------------------+---------------------+----------------+-------

In [15]:
df.count(), len(df.columns)

(160057, 92)

In [16]:
df.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 43438|
|    0|116619|
+-----+------+



In [17]:
feature_cols = df.columns[1:-1]
print(feature_cols)
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
assembler
df = assembler.transform(df)

['repeat_buy_prob_90d', 'avg_price_per_cheapest_common_unit', 'avg_price_per_most_common_unit', 'repeat_buy_prob_30d', 'marketshare_in_dep', 'share_of_cust_bought_prod', 'repeat_buy_prob_60d', 'marketshare_in_cat', 'competing_products_in_cat', 'cat_repeat_buy_prob_90d', 'share_of_cust_bought_cat', 'cat_repeat_buy_prob_60d', 'cat_repeat_buy_prob_30d', 'share_of_cust_bought_dep', 'offer_value', 'has_bought_category', 'has_bought_category_q', 'dep_count_30', 'has_bought_category_180', 'has_bought_category_90', 'has_bought_category_q_90', 'has_bought_category_q_180', 'has_bought_category_60', 'has_bought_category_q_60', 'has_bought_category_30', 'has_bought_category_q_30', 'has_bought_brand_180', 'has_bought_brand_90', 'has_bought_brand_q_180', 'has_bought_brand_q_90', 'established_product', 'never_bought_category', 'has_bought_brand_60', 'has_bought_brand_q_60', 'prodid_count_all', 'has_bought_brand', 'bought_product_before', 'has_bought_brand_company', 'has_bought_brand_company_category'

In [18]:
data = df.select(['features', 'label'])
data.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(90,[0,1,2,3,4,5,...|    1|
|(90,[0,1,2,3,4,5,...|    0|
|(90,[0,1,2,3,4,5,...|    1|
|(90,[0,1,2,3,4,5,...|    0|
|(90,[0,1,2,3,4,5,...|    1|
|(90,[0,1,2,3,4,5,...|    0|
|(90,[0,1,2,3,4,5,...|    1|
|(90,[0,1,2,3,4,5,...|    0|
|(90,[0,1,2,3,4,5,...|    0|
|(90,[0,1,2,3,4,5,...|    1|
+--------------------+-----+
only showing top 10 rows



In [19]:
data.select(['label']).distinct().show()

+-----+
|label|
+-----+
|    1|
|    0|
+-----+



In [20]:
X_train, X_test = data.randomSplit([0.80, 0.20], seed=22)

X_train.count(), len(X_train.columns)

(127967, 2)

In [21]:
X_test.count(), len(X_test.columns)

(32090, 2)

In [22]:
%%timeit
X_test.count(), len(X_test.columns)

1 loop, best of 5: 3.64 s per loop


In [23]:
import sys
import time
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor

In [24]:
rf = RandomForestClassifier(numTrees=100, maxDepth=15, labelCol="label", featuresCol='features', seed=42)
model = rf.fit(X_train)

In [ ]:
%%timeit
rf = RandomForestClassifier(numTrees=100, maxDepth=15, labelCol="label", featuresCol='features', seed=42)
model = rf.fit(X_train)

In [25]:
prediction = model.transform(X_test)
prediction.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(90,[0,1,2,3,4,5,...|    0|[80.8162350385840...|[0.80816235038584...|       0.0|
|(90,[0,1,2,3,4,5,...|    0|[91.3442607064456...|[0.91344260706445...|       0.0|
|(90,[0,1,2,3,4,5,...|    0|[87.1272870867306...|[0.87127287086730...|       0.0|
|(90,[0,1,2,3,4,5,...|    1|[73.9059336858146...|[0.73905933685814...|       0.0|
|(90,[0,1,2,3,4,5,...|    0|[71.3841433574657...|[0.71384143357465...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [26]:
# http://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html?highlight=classificationevaluator#pyspark.ml.evaluation.MulticlassClassificationEvaluator.metricName
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
rf_accuracy = evaluator.evaluate(prediction)
rf_accuracy

0.7527578684948583

In [29]:
rf_auc = BinaryClassificationEvaluator(
    labelCol='label').evaluate(prediction)
rf_auc 

0.712095303802491

# **testHistory**

In [30]:
test90 = spark.read.csv('/content/drive/MyDrive/project/data/test90.csv', header=True, inferSchema=True)
test90.show(1)

+---+-------------------+----------------------------------+------------------------------+-------------------+------------------+-------------------------+-------------------+------------------+-------------------------+-----------------------+------------------------+-----------------------+-----------------------+------------------------+-----------+-------------------+---------------------+------------+-----------------------+----------------------+------------------------+-------------------------+----------------------+------------------------+----------------------+------------------------+--------------------+-------------------+----------------------+---------------------+-------------------+---------------------+-------------------+---------------------+----------------+----------------+---------------------+------------------------+---------------------------------+------------------+-------------------------+-------------------+---------------------+----------------+-------

In [31]:
feature_cols = test90.columns[1:]
print(feature_cols)
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
assembler
test90 = assembler.transform(test90)

['repeat_buy_prob_90d', 'avg_price_per_cheapest_common_unit', 'avg_price_per_most_common_unit', 'repeat_buy_prob_30d', 'marketshare_in_dep', 'share_of_cust_bought_prod', 'repeat_buy_prob_60d', 'marketshare_in_cat', 'competing_products_in_cat', 'cat_repeat_buy_prob_90d', 'share_of_cust_bought_cat', 'cat_repeat_buy_prob_60d', 'cat_repeat_buy_prob_30d', 'share_of_cust_bought_dep', 'offer_value', 'has_bought_category', 'has_bought_category_q', 'dep_count_30', 'has_bought_category_180', 'has_bought_category_90', 'has_bought_category_q_90', 'has_bought_category_q_180', 'has_bought_category_60', 'has_bought_category_q_60', 'has_bought_category_30', 'has_bought_category_q_30', 'has_bought_brand_180', 'has_bought_brand_90', 'has_bought_brand_q_180', 'has_bought_brand_q_90', 'established_product', 'never_bought_category', 'has_bought_brand_60', 'has_bought_brand_q_60', 'prodid_count_all', 'has_bought_brand', 'bought_product_before', 'has_bought_brand_company', 'has_bought_brand_company_category'

In [1]:
testset = test90.select(['features'])

NameError: ignored

In [33]:
predict_test = model.transform(testset)
predict_test.show(5)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(90,[0,1,2,3,4,5,...|[69.9777506189891...|[0.69977750618989...|       0.0|
|(90,[0,1,2,3,4,5,...|[67.9634721055827...|[0.67963472105582...|       0.0|
|(90,[0,1,2,3,4,5,...|[79.7485436730133...|[0.79748543673013...|       0.0|
|(90,[0,1,2,3,4,5,...|[83.2889152321131...|[0.83288915232113...|       0.0|
|(90,[0,1,2,3,4,5,...|[73.0847890897709...|[0.73084789089770...|       0.0|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [34]:
predict_test.toPandas().to_csv('/content/drive/MyDrive/project/data/PREDICT_TEST.csv')

/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
